In [ ]:
import sys  
sys.path.insert(0, '..')

import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

from util.plots import show_plot

feature = pd.read_csv('../data/feature_nirspec100.csv').values
label = pd.read_csv('../data/label_nirspec100.csv').values

train_feature = feature[:50000]
train_label = label[:50000]
test_feature = feature[50000:]
test_label = label[50000:]

print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))
print(train_feature.shape, train_label.shape)

## Model cu straturi dense

In [ ]:
inputs = keras.Input(shape = (198,))

x = layers.Dense(150, activation='relu')(inputs)
x = layers.Dropout(0.1)(x)
x = layers.Dense(44, activation='relu')(x)

outputs = layers.Dense(12, activation='relu')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="d_model")

model.compile(
    optimizer=keras.optimizers.Nadam(),
    loss=keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)
model.summary()

EPOCHS = 95
history = model.fit(train_feature,
                    train_label,
                    epochs=EPOCHS, batch_size=200,
                    validation_split = 0.1)

model.save('models/d_model_100')

print('test', model.evaluate(test_feature, test_label, batch_size=5000))
print('train', model.evaluate(train_feature, train_label, batch_size=5000))

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
show_plot(model.predict(test_feature[:2500]), test_label[:2500],
          model.predict(train_feature[:5000]), train_label[:5000],
         (0.5, 0.5), 600)

In [ ]:
# Datele prezise cel mai bine

d = np.abs(model.predict(test_feature) - test_label)
p = np.argwhere(d.mean(axis=1) < 0.025).reshape([-1])
print(len(p))
f = test_feature[p]
l = test_label[p]
print(p[:500])
show_plot(model.predict(f[:500]), l[:500],
          model.predict(test_feature[:100]), test_label[:100],
         (0.4, 0.4), 800)

In [ ]:
# Date cu input identic dar cu output diferit.

p = np.argwhere(train_feature.sum(1)==0)
print(train_feature[p][0:2])
print(train_label[p].mean(0))
print('nr instante ce respecta proprietatea: ', len(p))

## Modele cu straturi convoluționale

In [ ]:
train_feature_2 = train_feature[:,:,None]
test_feature_2 = test_feature[:,:,None]
print(train_feature_2.shape, test_feature_2.shape)

In [ ]:
inputs = keras.Input(shape = (198,1))

x = layers.Conv1D(32, 3, activation='relu')(inputs)
x = layers.Flatten()(x)

y = layers.Conv1D(16, 3, activation='relu')(inputs)
y = layers.MaxPooling1D(16, 3)(y)
y = layers.Conv1D(16, 3, activation='relu')(y)
y = layers.MaxPooling1D(16, 3)(y)
y = layers.Flatten()(y)

x = layers.Dense(300, activation='relu')(layers.concatenate([x,y]))
x = layers.Dropout(0.3)(x)
x = layers.Dense(60, activation='relu')(x)
x = layers.Dense(60, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(40, activation='relu')(x)

outputs = layers.Dense(12, activation='relu')(x)


cnn_model = keras.Model(inputs=inputs, outputs=outputs, name="cnn_model")

cnn_model.compile(
    optimizer=keras.optimizers.Nadam(),
    loss=keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)
cnn_model.summary()

EPOCHS = 100
history = cnn_model.fit(train_feature_2,
                        train_label,
                        epochs=EPOCHS, batch_size=200,
                        validation_split = 0.1)

cnn_model.save('models/cnn_model_100')

print('test', cnn_model.evaluate(test_feature_2, test_label, batch_size=2500))
print('train', cnn_model.evaluate(train_feature_2, train_label, batch_size=2500))

In [ ]:
show_plot(cnn_model.predict(test_feature_2[:2500]), test_label[:2500],
          cnn_model.predict(train_feature_2[:5000]), train_label[:5000],
         (0.5, 0.5), 600)

In [ ]:
def inception_layers(input_layer):
    a = layers.Conv1D(64, 1, activation='relu', padding='same')(input_layer)
    
    b = layers.Conv1D(16, 1, activation='relu', padding='same')(input_layer)
    c = layers.Conv1D(16, 3, activation='relu', padding='same')(b)
    
    d = layers.Conv1D(16, 1, activation='relu', padding='same')(input_layer)
    e = layers.Conv1D(16, 5, activation='relu', padding='same')(d)
    
    f = layers.MaxPooling1D(3, strides=1, padding='same')(input_layer)
    g = layers.Conv1D(16, 1, activation='relu', padding='same')(f)
    
    return layers.concatenate([a, c, e, g], axis=2)

inputs = keras.Input(shape = (198,1))
x = inception_layers(inputs)
x = inception_layers(x)
x = layers.Flatten()(x)

x = layers.Dropout(rate=0.5)(x)
x = layers.Dense(400, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(150, activation='relu')(x)

outputs = layers.Dense(12, activation='relu')(x)


inception_model = keras.Model(inputs=inputs, outputs=outputs, name="inception_model")

inception_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)
inception_cnn_model.summary()

EPOCHS = 100
history = inception_model.fit(train_feature_2,
                              train_label,
                              epochs=EPOCHS, batch_size=200,
                              validation_split = 0.1)

inception_model.save('models/inception_model_100')

print('test', inception_model.evaluate(test_feature_2, test_label, batch_size=2500))
print('train', inception_model.evaluate(train_feature_2, train_label, batch_size=2500))

In [ ]:
show_plot(inception_model.predict(test_feature_2[:2500]), test_label[:2500],
          inception_model.predict(train_feature_2[:5000]), train_label[:5000],
         (0.5, 0.5), 600)